In [ ]:
import numpy as np
import pandas as pd
import nibabel as nib
import itertools
import time
import os

In [ ]:
def before_after(a, total_width):
    before = (total_width-a)//2
    after = total_width-before-a
    return (before, after)


def get_pad_width(x,y,z,total_width=400):
    return (before_after(x,total_width),
            before_after(y,total_width),
            before_after(z,total_width))


def get_padded_data(data, width=400):
    x,y,z = list(data.shape)
    return np.pad(data,get_pad_width(x,y,z),'constant',constant_values=0)

In [ ]:
df = pd.read_csv('../csvs/GT_Training.csv', header=None, names=['file','label'])
df.file = df.file.map(lambda x: x.replace("'",''))
df['img'] = df.file.map(lambda x: nib.load('../data/' + str(x) + '.nii.gz'))

In [ ]:
start = time.time()
start100 = start
for i, img in enumerate(df.img):
    if i % 5 == 0 and i != 0:
        now = time.time()
        print('i = {:<10d}time for last 5 = {:<10.3g}total time = {:<10.3g}'.format(i,now-start100,now-start))
        start100 = time.time()
    export_path = '../data_padded/' + str(df.file[i]) + '.nii.gz'
    if not os.path.isfile(export_path):
        padded = get_padded_data(img.get_fdata())
        padded_img = nib.Nifti1Image(padded, img.affine)
        nib.save(padded_img, export_path)

print('Done with {:d} rows. Total time = {:<10.3g}'.format(len(df),time.time()-start))